# Morphological Inflection in English

```
sleep <V;PST> -> slept
```

You can find out more about the annotation schema used for the morphosyntactic specification of the target forms [here](https://unimorph.github.io/doc/unimorph-schema.pdf).




In [1]:
!pip install -U accelerate
!pip install -U transformers
!pip install -U evaluate
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━

## Download datasets

In [80]:
import requests
from typing import List, Dict
from requests.exceptions import RequestException
from IPython.display import display, HTML
import time

def download_data(url: str, max_retries: int = 3, timeout: int = 10) -> List[str]:
    """
    Download and process data from a given URL.

    Args:
        url: The URL to download data from
        max_retries: Maximum number of retry attempts
        timeout: Timeout in seconds for the request

    Returns:
        List of strings, each representing a line from the downloaded data
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            data = response.text.strip().split('\n')
            print(f"✓ Successfully downloaded {len(data)} lines from {url.split('/')[-1]}")
            return data

        except RequestException as e:
            if attempt == max_retries - 1:
                display(HTML(f"<b style='color: red'>Error downloading {url.split('/')[-1]}: {str(e)}</b>"))
                raise
            print(f"⚠️ Attempt {attempt + 1} failed, retrying...")
            time.sleep(2 ** attempt)  # Exponential backoff

# Data URLs
train_data_path = 'https://raw.githubusercontent.com/sigmorphon/2022InflectionST/refs/heads/main/part2/eng_1000.train'
dev_data_path = 'https://raw.githubusercontent.com/sigmorphon/2022InflectionST/refs/heads/main/part2/eng.dev'

# Download and store data
data = {}
try:
    data['train'] = download_data(train_data_path)
    data['dev'] = download_data(dev_data_path)
except Exception as e:
    display(HTML(f"<b style='color: red'>Failed to load datasets: {str(e)}</b>"))

✓ Successfully downloaded 1000 lines from eng_1000.train
✓ Successfully downloaded 454 lines from eng.dev


In [81]:
# Display dataset statistics with detailed formatting
def display_data_summary(data: dict) -> None:
    """
    Display summary statistics and sample data in a formatted way.

    Args:
        data: Dictionary containing 'train' and 'dev' datasets
    """
    # Print dataset sizes with formatted output
    print(f"\n📊 Dataset Statistics:")
    print(f"  • Training samples: {len(data['train']):,}")
    print(f"  • Development samples: {len(data['dev']):,}")

    sample = data['train'][0]
    print(f"\n📝 Sample Data Format:", sample)

# Display the summary
display_data_summary(data)



📊 Dataset Statistics:
  • Training samples: 1,000
  • Development samples: 454

📝 Sample Data Format: take	took	V;PST


## Data preprocessing

In [82]:
import re
import regex
from typing import List, Tuple, Set, Dict

def parse_tag(tag: str) -> str:
    """
    Parse a tag string by removing unwanted characters and formatting each tag.

    Args:
        tag: Input tag string containing tags separated by delimiters

    Returns:
        Formatted string with each tag wrapped in angle brackets
    """
    tag = re.sub(r"\)|\(|,|;", ' ', tag).split()
    return ''.join(['<{}>'.format(t) for t in tag])


def preprocess_data(raw_data: List[str]) -> List[Tuple[str, str]]:
    """
    Preprocess raw data by parsing each line into formatted input-target pairs.

    Args:
        raw_data: List of raw data lines (lemma, target, tag separated by tabs)

    Returns:
        List of tuples containing (formatted_input, target)
    """
    preprocessed_data = []
    for line in raw_data:
        try:
            lemma, target, tag = line.split('\t')
            formatted_input = f"{lemma} {parse_tag(tag)}"
            preprocessed_data.append((formatted_input, target))
        except ValueError as e:
            print(f"⚠️ Skipping malformed line: {line} (Error: {e})")

    return preprocessed_data

def extract_vocab(data: List[Tuple[str, str]]) -> Tuple[Set[str], List[str], Dict[str, int]]:
    """
    Extract vocabulary, tags, and character mappings from preprocessed data.

    Args:
        data: List of preprocessed (input, target) pairs

    Returns:
        Tuple containing:
        - Set of unique characters
        - List of unique tags
        - Dictionary mapping characters to indices
    """
    # Extract characters from lemmas and targets
    chars = set(''.join([d[0].split()[0] + d[1] for d in data]))

    # Create character to index mapping
    char2id = {char: idx for idx, char in enumerate(sorted(chars))}

    # Extract unique tags
    tags = list(set(regex.findall(r"<[A-Za-z0-9]+>", ' '.join(d[0] for d in data))))

    return chars, tags, char2id

# Process the data
data['train'] = preprocess_data(data['train'])
data['dev'] = preprocess_data(data['dev'])

# Display sample of preprocessed data
print(f"\n📝 Preprocessed sample:")
print(f"  {data['train'][0]}")

# Extract vocabulary and mappings
chars, tags, char2id = extract_vocab(data['train'])



📝 Preprocessed sample:
  ('take <V><PST>', 'took')

📊 Vocabulary Statistics:
  • Unique characters: 26
  • Unique tags: 2


In [84]:
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Tuple, Dict, List, Union
from transformers import PreTrainedTokenizer
from transformers.tokenization_utils import AddedToken
import json
import warnings

@dataclass
class TokenizerConfig:
    """Configuration for CustomTokenizer"""
    bos_token: str = "<s>"
    eos_token: str = "</s>"
    unk_token: str = "<unk>"
    pad_token: str = "<pad>"
    max_len: int = 512

class CustomTokenizer(PreTrainedTokenizer):
    """
    Custom tokenizer for character-level tokenization with special handling for morphosyntactic features.

    Processes input and output word forms character by character while treating morphosyntactic
    features as special atomic tokens.
    """

    model_input_names = ["input_ids", "attention_mask"]

    def __init__(
        self,
        vocab: Dict[str, int],
        additional_special_tokens: Optional[List[str]] = None,
        max_len: int = 512,
        **kwargs
    ) -> None:
        """
        Initialize the tokenizer.

        Args:
            vocab: Dictionary mapping characters to their IDs
            additional_special_tokens: List of special tokens (e.g., morphological tags)
            max_len: Maximum sequence length
        """
        config = TokenizerConfig()

        # Initialize token mappings
        self.__token_ids = vocab.copy()
        self.__id_tokens = {v: k for k, v in vocab.items()}

        # Process special tokens
        special_tokens = {
            'pad_token': self._create_added_token(config.pad_token),
            'bos_token': self._create_added_token(config.bos_token),
            'eos_token': self._create_added_token(config.eos_token),
            'unk_token': self._create_added_token(config.unk_token)
        }

        # Initialize special token decoder
        self._added_tokens_decoder = {
            i: token for i, token in enumerate(special_tokens.values())
        }
        self.offset = len(self._added_tokens_decoder)

        super().__init__(
            **special_tokens,
            additional_special_tokens=additional_special_tokens,
            max_len=max_len,
            **kwargs
        )

    @staticmethod
    def _create_added_token(token: Union[str, AddedToken]) -> AddedToken:
        """Create an AddedToken with consistent settings."""
        return AddedToken(token, lstrip=False, rstrip=False) if isinstance(token, str) else token

    @property
    def vocab_size(self) -> int:
        """Get the size of the vocabulary."""
        return len(self.__token_ids)

    def get_vocab(self) -> Dict[str, int]:
        """Get the full vocabulary including special tokens."""
        vocab = {
            self.convert_ids_to_tokens(i): i
            for i in range(self.vocab_size + self.offset)
        }
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _add_eos(self, token_ids: List[int]) -> List[int]:
        """Add end-of-sequence token to the token list."""
        return token_ids + [self.eos_token_id]

    def create_token_type_ids_from_sequences(
        self,
        token_ids_0: List[int],
        token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """Create token type IDs for single or paired sequences."""
        total_length = len(token_ids_0) + 1  # +1 for EOS
        if token_ids_1:
            total_length += len(token_ids_1) + 1
        return [0] * total_length

    def build_inputs_with_special_tokens(
        self,
        token_ids_0: List[int],
        token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """Build model inputs by adding special tokens."""
        token_ids_0 = self._add_eos(token_ids_0)
        if token_ids_1 is None:
            return token_ids_0
        return token_ids_0 + self._add_eos(token_ids_1)

    def _tokenize(self, text: str, **kwargs) -> List[str]:
        """Tokenize text into characters."""
        return list(text)

    def _convert_token_to_id(self, token: str) -> int:
        """Convert a token to its ID."""
        return (self.__token_ids.get(token, self.unk_token_id - self.offset)
                + self.offset)

    def _convert_id_to_token(self, index: int) -> str:
        """Convert an ID back to its token."""
        adjusted_index = index - self.offset
        return self.__id_tokens.get(adjusted_index, self.unk_token)

    def convert_tokens_to_string(self, tokens: List[str]) -> str:
        """Convert tokens back to string."""
        return "".join(tokens)

    def save_vocabulary(self,
                       save_directory: str,
                       filename_prefix: Optional[str] = None) -> Tuple[str]:
        """Save the vocabulary to a file."""
        prefix = filename_prefix or ""
        vocab_path = Path(save_directory) / f"{prefix}vocab.json"

        with vocab_path.open('w', encoding='utf-8') as f:
            json.dump(self.__token_ids, f, ensure_ascii=False, indent=2)

        return (str(vocab_path),)

# Initialize tokenizer with vocabulary and tags
tokenizer = CustomTokenizer(char2id, additional_special_tokens=tags, max_len=100)

# Display example tokenization
sample_idx = 54
sample_text = data['train'][sample_idx][0]
tokens = tokenizer.tokenize(sample_text)
print(f"Tokenization Example:")
print(f"Input text: {sample_text}")
print(f"Tokens: {tokens}")


Tokenization Example:
Input text: shut <V><PST>
Tokens: ['s', 'h', 'u', 't', ' ', '<V>', '<PST>']


## Training T5 model ([Raffel et al.](https://arxiv.org/pdf/1910.10683))

In [85]:
from transformers import T5Config, T5ForConditionalGeneration
from typing import Optional

def create_t5_model(
    tokenizer,
    d_ff: int = 1024,
    d_model: int = 256,
    num_layers: int = 4,
    num_heads: int = 4,
    dropout_rate: float = 0.2,
    max_new_tokens: int = 32,
    d_kv: Optional[int] = None
) -> T5ForConditionalGeneration:
    """
    Creates and configures a T5 model for morphological generation.

    Args:
        tokenizer: Custom tokenizer for the model
        d_ff: Dimension of the feed-forward layer
        d_model: Dimension of the model embeddings
        num_layers: Number of encoder and decoder layers
        num_heads: Number of attention heads
        dropout_rate: Dropout rate for regularization
        max_new_tokens: Maximum number of tokens to generate
        d_kv: Dimension of key and value vectors (defaults to d_model // num_heads)

    Returns:
        Configured T5ForConditionalGeneration model
    """
    # Calculate default d_kv if not provided
    if d_kv is None:
        d_kv = d_model // num_heads

    # Configure model architecture
    config = T5Config(
        d_ff=d_ff,
        d_model=d_model,
        d_kv=d_kv,
        num_layers=num_layers,
        num_decoder_layers=num_layers,  # Match encoder layers
        num_heads=num_heads,
        dropout_rate=dropout_rate,
        vocab_size=len(tokenizer),
        is_encoder_decoder=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id
    )

    # Initialize model
    model = T5ForConditionalGeneration(config)

    # Configure generation parameters
    model.config.decoder_start_token_id = tokenizer.bos_token_id
    model.generation_config.decoder_start_token_id = tokenizer.bos_token_id
    model.generation_config.max_new_tokens = max_new_tokens
    model.generation_config.eos_token_id = tokenizer.eos_token_id
    model.generation_config.no_repeat_ngram_size = 2
    model.generation_config.length_penalty = 1.0
    model.generation_config.num_beams = 10

    return model

# Initialize the model with default parameters
model = create_t5_model(tokenizer)


In [86]:
from torch.utils.data import Dataset
from typing import List, Dict, Tuple, Union
import numpy as np
import torch

class CustomDataset(Dataset):
    """
    Custom dataset for morphological generation task.
    Handles tokenization of input lemmas and target forms.
    """

    def __init__(
        self,
        data: List[Tuple[str, str]],
        tokenizer,
        max_length: int = 128
    ) -> None:
        """
        Initialize dataset.

        Args:
            data: List of (input, target) string pairs
            tokenizer: Tokenizer for encoding text
            max_length: Maximum sequence length
        """
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        """Return dataset size."""
        return len(self.data)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Get tokenized item from dataset.

        Args:
            idx: Index of item to retrieve

        Returns:
            Dictionary with input_ids and labels tensors
        """
        input_text, target_text = self.data[idx]

        # Tokenize input and target
        model_inputs = self.tokenizer(
            input_text,
            padding='longest',
            truncation=True,
            max_length=self.max_length,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = self.tokenizer(
            target_text,
            padding='longest',
            truncation=True,
            max_length=self.max_length,
            add_special_tokens=True,
            return_tensors='pt'
        )

        return {
            "input_ids": model_inputs["input_ids"].squeeze(0),
            "attention_mask": model_inputs["attention_mask"].squeeze(0),
            "labels": labels["input_ids"].squeeze(0)
        }

def postprocess_data(
    token_ids: Union[np.ndarray, torch.Tensor],
    tokenizer
) -> List[str]:
    """
    Post-process token IDs into readable text.

    Args:
        token_ids: Array/Tensor of token IDs
        tokenizer: Tokenizer for decoding

    Returns:
        List of decoded strings with special tokens removed
    """
    # Convert torch tensor to numpy if needed
    if isinstance(token_ids, torch.Tensor):
        token_ids = token_ids.cpu().numpy()

    # Replace padding indices
    token_ids = np.where(token_ids != -100, token_ids, tokenizer.pad_token_id)

    # Decode tokens to strings
    return tokenizer.batch_decode(token_ids, skip_special_tokens=True)


In [89]:
"""
Evaluation metrics for morphological generation task.
Includes exact match scoring and detailed error analysis.
"""

import evaluate
import numpy as np
from typing import Dict, List, Tuple, Any
import random
from dataclasses import dataclass
import Levenshtein
from collections import defaultdict

@dataclass
class PredictionExample:
    """Store prediction examples for analysis"""
    input: str
    prediction: str
    target: str
    is_correct: bool
    edit_distance: int

class MetricsComputer:
    """Compute and analyze model predictions"""

    def __init__(self, tokenizer, sample_size: int = 15):
        """
        Initialize metrics computer.

        Args:
            tokenizer: Tokenizer for decoding predictions
            sample_size: Number of random examples to sample for analysis
        """
        self.metric = evaluate.load("exact_match")
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.error_stats = defaultdict(int)

    def compute_edit_distance(self, pred: str, target: str) -> int:
        """Compute Levenshtein distance between prediction and target"""
        return Levenshtein.distance(pred, target)

    def analyze_predictions(
        self,
        decoded_preds: List[str],
        decoded_labels: List[str],
        inputs: List[str] = None
    ) -> List[PredictionExample]:
        """
        Analyze a sample of predictions.

        Args:
            decoded_preds: List of model predictions
            decoded_labels: List of target labels
            inputs: Optional list of input texts

        Returns:
            List of PredictionExample objects
        """
        sample_indices = random.sample(range(len(decoded_preds)),
                                     min(self.sample_size, len(decoded_preds)))

        examples = []
        for idx in sample_indices:
            pred = decoded_preds[idx]
            target = decoded_labels[idx]
            input_text = inputs[idx] if inputs else ""

            example = PredictionExample(
                input=input_text,
                prediction=pred,
                target=target,
                is_correct=pred == target,
                edit_distance=self.compute_edit_distance(pred, target)
            )
            examples.append(example)

            # Track error types
            if not example.is_correct:
                self.error_stats['total_errors'] += 1
                if len(pred) != len(target):
                    self.error_stats['length_mismatch'] += 1
                if pred == target[::-1]:
                    self.error_stats['reversed'] += 1

        return examples

    def compute_metrics(self, eval_preds: Tuple[np.ndarray, np.ndarray]) -> Dict[str, float]:
        """
        Compute evaluation metrics for predictions.

        Args:
            eval_preds: Tuple of (predictions, labels) arrays

        Returns:
            Dictionary of metric scores
        """
        preds, labels = eval_preds
        decoded_preds = postprocess_data(preds, self.tokenizer)
        decoded_labels = postprocess_data(labels, self.tokenizer)

        # Compute exact match score
        exact_match = self.metric.compute(
            predictions=decoded_preds,
            references=decoded_labels
        )

        # Analyze sample predictions
        examples = self.analyze_predictions(decoded_preds, decoded_labels)

        # Compute additional metrics
        edit_distances = [ex.edit_distance for ex in examples]

        metrics = {
            "exact_match": exact_match["exact_match"],
            "avg_edit_distance": np.mean(edit_distances),
            "max_edit_distance": max(edit_distances),
            "error_rate": self.error_stats['total_errors'] / len(examples)
        }

        # Print analysis examples
        self._print_analysis(examples)

        return metrics

    def _print_analysis(self, examples: List[PredictionExample]) -> None:
        """Print detailed analysis of prediction examples"""
        print("\n📊 Prediction Analysis:")
        print(f"Analyzing {len(examples)} random examples...")
        print("\nDetailed Examples:")

        for i, ex in enumerate(examples, 1):
            status = "✓" if ex.is_correct else "✗"
            print(f"\n{i}. {status} Input: {ex.input}")
            print(f"   Prediction: {ex.prediction}")
            print(f"   Target: {ex.target}")
            print(f"   Edit Distance: {ex.edit_distance}")

        print("\nError Statistics:")
        for error_type, count in self.error_stats.items():
            print(f"  • {error_type}: {count}")

# Initialize metrics computer
metrics_computer = MetricsComputer(tokenizer)

# Use for evaluation
compute_metrics = metrics_computer.compute_metrics


In [98]:
import warnings
from transformers import utils

# Monkey patch the deprecated warning in transformers utils
utils.deprecation_warning = lambda *args, **kwargs: None

from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from pathlib import Path

# Create datasets with proper batch size
dataset = {
    'train': CustomDataset(data['train'], tokenizer, max_length=128),
    'dev': CustomDataset(data['dev'], tokenizer, max_length=128)
}

# Initialize data collator for sequence-to-sequence task
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    label_pad_token_id=tokenizer.pad_token_id
)

# Define training arguments with optimized parameters
training_args = Seq2SeqTrainingArguments(
    output_dir=Path('model_checkpoints'),
    max_steps=1000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    lr_scheduler_type='inverse_sqrt',
    warmup_steps=2000,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    label_smoothing_factor=0.1,

    # Evaluation settings
    evaluation_strategy="steps",
    eval_steps=200,
    eval_delay=200,
    metric_for_best_model='exact_match',

    # Checkpointing
    save_strategy="steps",
    save_steps=200,
    save_total_limit=3,  # Keep more checkpoints

    # Generation settings
    gradient_accumulation_steps=4,
    predict_with_generate=True,
    generation_num_beams=5,

    fp16=True,
    gradient_checkpointing=True,

    # Logging
    logging_strategy="steps",
    logging_steps=100,
    logging_first_step=True,

    # Other settings
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    report_to="none",
    seed=42,
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Step,Training Loss,Validation Loss,Exact Match,Avg Edit Distance,Max Edit Distance,Error Rate
200,1.119600,1.191470,0.151982,2.266667,6,15.666667
400,1.196100,1.175861,0.151982,2.000000,6,16.333333
600,1.183400,1.162995,0.167401,2.466667,5,17.200000
800,1.152100,1.146829,0.176211,2.666667,4,18.133333
1000,1.121300,1.138121,0.176211,2.733333,6,19.000000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr


📊 Prediction Analysis:
Analyzing 15 random examples...

Detailed Examples:

1. ✗ Input: N/A
   Prediction: iled
   Target: lay
   Edit Distance: 3

2. ✗ Input: N/A
   Prediction: caked
   Target: jacked
   Edit Distance: 2

3. ✓ Input: N/A
   Prediction: shoveled
   Target: shoveled
   Edit Distance: 0

4. ✗ Input: N/A
   Prediction: shiged
   Target: sighed
   Edit Distance: 2

5. ✗ Input: N/A
   Prediction: stramed
   Target: mastered
   Edit Distance: 5

6. ✗ Input: N/A
   Prediction: grated
   Target: regurgitated
   Edit Distance: 6

7. ✗ Input: N/A
   Prediction: inted
   Target: invited
   Edit Distance: 2

8. ✗ Input: N/A
   Prediction: shoped
   Target: shod
   Edit Distance: 2

9. ✗ Input: N/A
   Prediction: bamed
   Target: bammed
   Edit Distance: 1

10. ✗ Input: N/A
   Prediction: dasted
   Target: devastated
   Edit Distance: 4

11. ✓ Input: N/A
   Prediction: hooked
   Target: hooked
   Edit Distance: 0

12. ✓ Input: N/A
   Prediction: hated
   Target: hated
   Edit Dis

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr


📊 Prediction Analysis:
Analyzing 15 random examples...

Detailed Examples:

1. ✓ Input: N/A
   Prediction: punched
   Target: punched
   Edit Distance: 0

2. ✗ Input: N/A
   Prediction: shased
   Target: sashayed
   Edit Distance: 3

3. ✗ Input: N/A
   Prediction: furthed
   Target: furthered
   Edit Distance: 2

4. ✗ Input: N/A
   Prediction: strated
   Target: starred
   Edit Distance: 3

5. ✗ Input: N/A
   Prediction: waded
   Target: waddled
   Edit Distance: 2

6. ✗ Input: N/A
   Prediction: prished
   Target: whispered
   Edit Distance: 5

7. ✗ Input: N/A
   Prediction: briged
   Target: bridged
   Edit Distance: 1

8. ✗ Input: N/A
   Prediction: magled
   Target: gleamed
   Edit Distance: 5

9. ✗ Input: N/A
   Prediction: claned
   Target: cleaned
   Edit Distance: 1

10. ✗ Input: N/A
   Prediction: compliced
   Target: compiled
   Edit Distance: 2

11. ✓ Input: N/A
   Prediction: cawed
   Target: cawed
   Edit Distance: 0

12. ✗ Input: N/A
   Prediction: panded
   Target: kidn

...


📊 Prediction Analysis:
Analyzing 15 random examples...

Detailed Examples:

1. ✗ Input: N/A
   Prediction: diged
   Target: dug
   Edit Distance: 3

2. ✗ Input: N/A
   Prediction: structed
   Target: structured
   Edit Distance: 2

3. ✗ Input: N/A
   Prediction: dasted
   Target: devastated
   Edit Distance: 4

4. ✗ Input: N/A
   Prediction: fruthed
   Target: furthered
   Edit Distance: 4

5. ✗ Input: N/A
   Prediction: ked
   Target: keyed
   Edit Distance: 2

6. ✗ Input: N/A
   Prediction: pized
   Target: zipped
   Edit Distance: 3

7. ✗ Input: N/A
   Prediction: snded
   Target: sent
   Edit Distance: 4

8. ✗ Input: N/A
   Prediction: prigled
   Target: privileged
   Edit Distance: 4

9. ✓ Input: N/A
   Prediction: munched
   Target: munched
   Edit Distance: 0

10. ✗ Input: N/A
   Prediction: shablisted
   Target: established
   Edit Distance: 3

11. ✗ Input: N/A
   Prediction: manded
   Target: demanded
   Edit Distance: 2

12. ✗ Input: N/A
   Prediction: wrinked
   Target: wri

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr


📊 Prediction Analysis:
Analyzing 15 random examples...

Detailed Examples:

1. ✗ Input: N/A
   Prediction: hared
   Target: heard
   Edit Distance: 2

2. ✗ Input: N/A
   Prediction: rented
   Target: entered
   Edit Distance: 3

3. ✗ Input: N/A
   Prediction: grined
   Target: grinned
   Edit Distance: 1

4. ✗ Input: N/A
   Prediction: waned
   Target: yawned
   Edit Distance: 2

5. ✗ Input: N/A
   Prediction: witched
   Target: twitched
   Edit Distance: 1

6. ✓ Input: N/A
   Prediction: fenced
   Target: fenced
   Edit Distance: 0

7. ✗ Input: N/A
   Prediction: mprented
   Target: experimented
   Edit Distance: 5

8. ✗ Input: N/A
   Prediction: cofered
   Target: forced
   Edit Distance: 4

9. ✗ Input: N/A
   Prediction: ivicled
   Target: civilized
   Edit Distance: 4

10. ✗ Input: N/A
   Prediction: mized
   Target: jimmied
   Edit Distance: 4

11. ✗ Input: N/A
   Prediction: worthed
   Target: threw
   Edit Distance: 5

12. ✓ Input: N/A
   Prediction: created
   Target: created


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1000, training_loss=1.1623030591011048, metrics={'train_runtime': 444.0954, 'train_samples_per_second': 288.226, 'train_steps_per_second': 2.252, 'total_flos': 74702934736896.0, 'train_loss': 1.1623030591011048, 'epoch': 125.0})

## Model Evaluation and Analysis

In [99]:
from typing import Dict, List
import time
from tqdm import tqdm

def evaluate_model(
    trainer,
    test_path: str,
    download_data,
    preprocess_data,
    tokenizer,
    batch_size: int = 32
) -> Dict[str, float]:
    """
    Evaluate model performance on test data with detailed metrics.

    Args:
        trainer: Seq2SeqTrainer instance
        test_path: URL to test data
        batch_size: Batch size for evaluation
    """
    try:
        # Load and preprocess test data
        print("Loading test data...")
        test_data = download_data(test_path)
        processed_data = preprocess_data(test_data)
        test_dataset = CustomDataset(processed_data, tokenizer)

        # Run evaluation
        print("Running evaluation...")
        start_time = time.time()
        result = trainer.evaluate(
            test_dataset,
            max_length=128,
            num_beams=5,
            metric_key_prefix="test"
        )
        eval_time = time.time() - start_time

        # Add additional metrics
        result.update({
            "test_samples": len(test_dataset),
            "eval_time_seconds": round(eval_time, 2),
            "eval_samples_per_second": round(len(test_dataset) / eval_time, 2)
        })

        # Print detailed results
        print("\n📈 Evaluation Results:")
        for metric, value in result.items():
            print(f"  • {metric}: {value:.4f}")

        return result

    except Exception as e:
        print(f"❌ Evaluation failed: {str(e)}")
        raise

# Run evaluation
test_data_path = 'https://raw.githubusercontent.com/sigmorphon/2022InflectionST/refs/heads/main/part2/eng_200.train'
evaluation_results = evaluate_model(
    trainer=trainer,
    test_path=test_data_path,
    download_data=download_data,
    preprocess_data=preprocess_data,
    tokenizer=tokenizer
)

# Store test accuracy
test_accuracy = evaluation_results['test_exact_match']


Loading test data...


Both `max_new_tokens` (=32) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✓ Successfully downloaded 200 lines from eng_200.train
Running evaluation...


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Both `max_new_tokens` (=32) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Both `max_new_tokens` (=32) and `max_length`(=128) seem to have


📊 Prediction Analysis:
Analyzing 15 random examples...

Detailed Examples:

1. ✗ Input: N/A
   Prediction: kiled
   Target: liked
   Edit Distance: 2

2. ✗ Input: N/A
   Prediction: sticked
   Target: stuck
   Edit Distance: 3

3. ✓ Input: N/A
   Prediction: fired
   Target: fired
   Edit Distance: 0

4. ✗ Input: N/A
   Prediction: prisured
   Target: surprised
   Edit Distance: 5

5. ✓ Input: N/A
   Prediction: crunched
   Target: crunched
   Edit Distance: 0

6. ✗ Input: N/A
   Prediction: snaped
   Target: snapped
   Edit Distance: 1

7. ✗ Input: N/A
   Prediction: chated
   Target: taught
   Edit Distance: 6

8. ✗ Input: N/A
   Prediction: conked
   Target: knocked
   Edit Distance: 3

9. ✗ Input: N/A
   Prediction: sted
   Target: set
   Edit Distance: 2

10. ✓ Input: N/A
   Prediction: stapled
   Target: stapled
   Edit Distance: 0

11. ✗ Input: N/A
   Prediction: ecited
   Target: excited
   Edit Distance: 1

12. ✓ Input: N/A
   Prediction: wished
   Target: wished
   Edit Dist